## 5.6 Carsy

Aplikacja wykorzystuje `BottomNavigation` oraz `DrawerNavigation`. Po raz kolejny wykorzystamy tylko statyczne dostarczone przez obiekt `dataProvider`. Wykorzystamy `RecyclerView` w kilku różnych konfiguracjach. Będzie to uboga wersja [Fuelio](https://play.google.com/store/apps/details?id=com.kajda.fuelio&hl=pl&gl=US)
- `RecyclerView` z grupowaniem elementów osiągniętym przez wykorzystanie dwóch `ViewHolder`

<table><tr><td><img src="https://media2.giphy.com/media/0lpf97nejk58XmgAP1/giphy.gif?cid=790b7611c2da1160570c8dd99550ec3cec763c3d4c38b025&rid=giphy.gif&ct=g" width="200" /></td><td><img src="https://media2.giphy.com/media/hv5Ewg1qBwSj99Ju82/giphy.gif?cid=790b7611ccbb9fccb551bafa99c524b39c01a03d957a334b&rid=giphy.gif&ct=g" width="200" /></td><td><img src="https://media4.giphy.com/media/FkSzT1Y6TNOtg89XKr/giphy.gif?cid=790b76119c5e65885f7be2c85549b516585eebeb4a8d4baa&rid=giphy.gif&ct=g" width="200" /></td></tr></table>

### **BottomNavigation**

Rozpocznijmy od dodania trzech pustych fragmentów (`OverviewFragment`, `CalculatorsFragment`, `TimeLineFragment`) i nawigacji

`navigation.xml`
```xml
<?xml version="1.0" encoding="utf-8"?>
<navigation xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    xmlns:tools="http://schemas.android.com/tools"
    android:id="@+id/navigation"
    app:startDestination="@id/overviewFragment">

    <fragment
        android:id="@+id/calculatorsFragment"
        android:name="pl.udu.uwr.pum.carsyappkotlin.fragments.CalculatorsFragment"
        android:label="fragment_calculators"
        tools:layout="@layout/fragment_calculators" />
    <fragment
        android:id="@+id/timeLineFragment"
        android:name="pl.udu.uwr.pum.carsyappkotlin.fragments.TimeLineFragment"
        android:label="fragment_time_line"
        tools:layout="@layout/fragment_time_line" />
    <fragment
        android:id="@+id/overviewFragment"
        android:name="pl.udu.uwr.pum.carsyappkotlin.fragments.OverviewFragment"
        android:label="fragment_overview"
        tools:layout="@layout/fragment_overview" />
</navigation>
```

Po raz kolejny dodajemy tylko fragmenty nie definiując dla nich akcji. `OverviewFragment` oznaczam jako fragment domowy. Musimy również utworzyć `menu` dla `BottomNavigation`

`bottom_menu.xml`
```xml
<?xml version="1.0" encoding="utf-8"?>
<menu xmlns:android="http://schemas.android.com/apk/res/android">
    <item
        android:id="@id/overviewFragment"
        android:title="@string/overview" />
    <item
        android:id="@id/timeLineFragment"
        android:title="@string/time_line" />

    <item
        android:id="@id/calculatorsFragment"
        android:title="@string/calculators" />
</menu>
```

Pamiętajmy, że `android:id` muszą być takie same jak w `navigation.xml` aby automatyczna nawigacja działała prawidłowo.

Przejdźmy do `MainActivity` i dodajmy nawigację oraz połączmy ją z `BottomNavigation`

In [ ]:
class MainActivity : AppCompatActivity() {

    private val navController: NavController by lazy {
        val navHostFragment = supportFragmentManager.findFragmentById(R.id.nav_host_fragment)
                as NavHostFragment
        navHostFragment.findNavController()
    }

    override fun onCreate(savedInstanceState: Bundle?) {
        super.onCreate(savedInstanceState)
        setContentView(R.layout.activity_main)

        val bottomNav = findViewById<BottomNavigationView>(R.id.bottom_nav_view)
        bottomNav?.setupWithNavController(navController)
    }
}

W efekcie powinniśmy otrzymać podstawową nawigację z trzema ekranami.

<img src="https://media1.giphy.com/media/x0j1SX0qmejSUNRtHu/giphy.gif?cid=790b7611bcaddd3fe77b775f20dd38e5e0cfa788c1e83df7&rid=giphy.gif&ct=g" width="150" />

### **TimeLineFragment**

Chcemy przygotować listę w postaci osi czasu, która będzie zbliżona do tego co oferuje aplikacja **Fuelio**

<img src="https://play-lh.googleusercontent.com/Lm6SO1jsUF-VjsrfkIV9x-qpPIkIOIvgWkBGmxyZuHWPSbkYMS0oLhgoBk9wLXq6Xw=w5120-h2880" width="150" />

Zrobimy to w postaci `RecyclerView` z dwoma `ViewHolder`. Wpierw przygotujmy dane. Typ kosztu zdefiniujemy jako `enum class`, będziemy przechowywać w niej typ oraz ikonę.

In [ ]:
enum class CostType(val costType: String, val icon: Int) {
    REFUELING("Tankowanie", R.drawable.ic_fuel),
    SERVICE("Serwis", R.drawable.ic_car_repair),
    PARKING("Parking", R.drawable.ic_parking),
    INSURANCE("Ubezpieczenie", R.drawable.ic_general_cost),
    TICKET("Mandat", R.drawable.ic_ticket)
}

Ikony standardowo wybieramy spośród dostępnych w **Android Studio** (**New -> Vector Asset**). Model danych zawiera typ kosztu, datę oraz kwotę.

In [ ]:
data class Cost (
    val type: CostType,
    val date: LocalDate,
    val amount: Int
)

Ponieważ będziemy posiadać pojedynczy `RecyclerView` i dwa `ViewHolder`, musimy w jakiś sposób rozróżnić typ naszych danych - dla uproszczenia będziemy sortować po miesiącach (bez uwzględnienia lat). Więc chcemy zdefiniować dwa typy 
- `CostListItem` - klasa bazowa

In [ ]:
open class CostListItem(
    val type: Int
) {
    companion object {
        const val TYPE_DATE = 0
        const val TYPE_GENERAL = 1
    }
}

- `CostDateItem` - posiadający tylko datę po której będziemy sortować oraz na podstawie której utworzymy wersję layoutu dla `RecyclerView`

In [ ]:
class CostDateItem(
    val date: String
) : CostListItem(TYPE_DATE)

- `CostGeneralItem` - każdy inny element dla którego również utworzymy layout

In [ ]:
class CostGeneralItem(
    val cost: Cost
) : CostListItem(TYPE_GENERAL)

Przejdźmy do obiektu `DataProvider` - w pierwszym kroku utworzymy listę kosztów

In [ ]:
private val generalCosts = List(100) {
    Cost(
        CostType.values()[Random.nextInt(CostType.values().size)],
        LocalDate.of(2022, Random.nextInt(1,13), Random.nextInt(1,28)),
        Random.nextInt(5000)
    )
}

Ponieważ mamy jeden `RecyclerView`, więc będziemy mieć jedną listę - musimy ją odpowiednio przygotować. Wykorzystamy `Map` i posortujemy całość po miesiącach, następnie pogrupujemy względem miesięcy.

In [ ]:
private val groupedCostMap: Map<String, List<Cost>> = generalCosts
    .sortedBy { it.date.month }
    .groupBy { it.date.month.toString() }

Następnie napiszemy funkcję `getTimeLineList` zwracająca listę dla `RecyclerView`

In [ ]:
fun getTimeLineList(): List<CostListItem>{

Utwórzmy listę pomocniczą

In [ ]:
val list = mutableListOf<CostListItem>()

Przechodzimy po wszystkich kluczach w naszej mapie

In [ ]:
for (date: String in groupedCostMap.keys) {

W pętli tworzę listę wszystkich elementów z danego miesiąca, następnie ją sortuję względem dnia miesiąca i każdy element `CostGeneralItem` dodaję do `list`

In [ ]:
val groupItems: List<Cost>? = groupedCostMap[date]
groupItems?.sortedBy { it.date.dayOfMonth }?.forEach {
    list.add(CostGeneralItem(it))
}

Ostatnim elementem pętli będzie dodanie `CostDateItem`

In [ ]:
list.add(CostDateItem(date))

Na koniec wychodzimy z pętli `for` i zwracamy listę

In [ ]:
    }
    return list
}

Pełna funkcja

In [ ]:
fun getTimeLineList(): List<CostListItem>{
    val list = mutableListOf<CostListItem>()
    for (date: String in groupedCostMap.keys) {
        val groupItems: List<Cost>? = groupedCostMap[date]
        groupItems?.sortedBy { it.date.dayOfMonth }?.forEach {
            list.add(CostGeneralItem(it))
        }
        list.add(CostDateItem(date))
    }
    return list
}

Podsumowując:
- tworzą listę do której wrzucę wszystkie elementy odpowiednio posortowane - lista zawiera elementy typu `CostListItem`
- przechodzę po kluczach wcześniej utworzonej mapy (miesiące)
- dodaję wszystkie elementy `CostGeneralItem` - wszystkie koszty dla danego miesiąca
- na koniec dodaję `CostDateItem` - miesiąc

Czyli tworzymy listę w odwrotnej kolejności - wpierw mamy wszystkie elementy dla danego miesiąca, po nim znajduje się element oznaczający dany miesiąc.

Przejdźmy do `TimeLineAdapter` - klasa będzie przyjmowała w parametrze `Context` i rozszerzała `RecyclerView.Adapter`

In [ ]:
class TimeLineAdapter(val context: Context) 
    : RecyclerView.Adapter<RecyclerView.ViewHolder>() {}

W pierwszej kolejności dodajmy listę elementów, ponieważ metoda `getTimeLineList` zwraca listę w odwrotnej kolejności, tutaj w pierwszym kroku wykonamy `reversed`

In [ ]:
private val itemList = DataProvider.getTimeLineList().reversed()

Zdefiniujmy dwie klasy `ViewHolder` - pierwszą będzie klasa dla elementu grupującego (tutaj jest to data - miesiąc)

In [ ]:
inner class DateViewHolder(view: View) : RecyclerView.ViewHolder(view) {

    private val dateTextView: TextView = view.findViewById(R.id.timeLineDateTextView)

    fun bind(item: CostDateItem) {
        dateTextView.text = item.date
    }
}

Dla tego `ViewHolder` definiujemy layout w pliku `date_item_timeline.xml`


```xml
<?xml version="1.0" encoding="utf-8"?>
<androidx.constraintlayout.widget.ConstraintLayout xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    android:layout_width="match_parent"
    android:layout_height="wrap_content"
    android:layout_marginStart="4dp"
    android:layout_marginEnd="4dp">

    <View
        android:id="@+id/RVcolorBarView"
        android:layout_width="16px"
        android:layout_height="0dp"
        android:layout_marginStart="24dp"
        android:background="?attr/colorPrimary"
        android:outlineAmbientShadowColor="?attr/colorPrimary"
        android:outlineSpotShadowColor="?attr/colorPrimary"
        app:layout_constraintBottom_toBottomOf="parent"
        app:layout_constraintStart_toStartOf="parent"
        app:layout_constraintTop_toTopOf="parent" />

    <ImageView
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:layout_marginStart="19dp"
        android:layout_marginTop="24dp"
        android:background="@drawable/ic_round_circle"
        android:backgroundTint="?attr/colorPrimary"
        android:contentDescription="@string/bullet"
        app:layout_constraintBottom_toBottomOf="@+id/RVcolorBarView"
        app:layout_constraintStart_toStartOf="parent"
        app:layout_constraintTop_toTopOf="@+id/RVcolorBarView" />

    <TextView
        android:id="@+id/timeLineDateTextView"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:layout_marginStart="16dp"
        android:layout_marginTop="24dp"
        android:fontFamily="serif"
        android:textColor="?attr/colorSecondary"
        android:padding="4dp"
        android:text="@string/date"
        android:textSize="16sp"
        app:layout_constraintBottom_toBottomOf="parent"
        app:layout_constraintStart_toStartOf="@id/RVcolorBarView"
        app:layout_constraintTop_toTopOf="parent" />

</androidx.constraintlayout.widget.ConstraintLayout>
```

Jak widzimy mamy tylko jedno pole `TextView` wyświetlające nazwę miesiąca po którym grupujemy. Pozostałe dwa elementy są graficznym indykatorem - `View` jest linią o zadanej grubości i `ImageView` tutaj ma formę wypełnionego koła.

Zdefiniujmy również drugi `ViewHolder` dla pozostałych elementów na liście - tutaj chcemy pokazać typ kosztu, kwotę, pełną datę, oraz ikonę dla odpowiedniego typu kosztu.

In [ ]:
inner class GeneralViewHolder (view: View) : RecyclerView.ViewHolder(view) {

    private val costTypeTextView: TextView = view.findViewById(R.id.timeLineCostTypeNameTextView)
    private val fullDateTextView: TextView = view.findViewById(R.id.timeLineFullDateTextView)
    private val amountTextView: TextView = view.findViewById(R.id.timeLineCostAmountTextView)
    private val iconImageView: ImageView = view.findViewById(R.id.iconTimeLineImageView)

    fun bind(item: CostGeneralItem) {
        costTypeTextView.text = item.cost.type.costType
        fullDateTextView.text = item.cost.date.format(dateFormatter)
        (item.cost.amount.toString() + " zł").also { amountTextView.text = it }
        iconImageView.background = ContextCompat.getDrawable(context, item.cost.type.icon)
    }
}

Będziemy również formatować nieco `String` przy dacie, więc zdefiniujmy `DateTimeFormatter`

In [ ]:
val dateFormatter: DateTimeFormatter = DateTimeFormatter.ofPattern("yyyy MMM dd")

Layout jest nieco bardziej skomplikowany

```xml
<?xml version="1.0" encoding="utf-8"?>
<androidx.constraintlayout.widget.ConstraintLayout xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    android:layout_width="match_parent"
    android:layout_height="wrap_content"
    android:layout_marginStart="4dp"
    android:layout_marginEnd="4dp">

    <View
        android:id="@+id/RVcolorBarView"
        android:layout_width="16px"
        android:layout_height="0dp"
        android:layout_marginStart="24dp"
        android:background="?attr/colorPrimary"
        android:outlineAmbientShadowColor="?attr/colorPrimary"
        android:outlineSpotShadowColor="?attr/colorPrimary"
        app:layout_constraintBottom_toBottomOf="parent"
        app:layout_constraintStart_toStartOf="parent"
        app:layout_constraintTop_toTopOf="parent" />

    <ImageView
        android:id="@+id/imageView"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:layout_marginStart="3.2dp"
        android:background="@drawable/ic_round_circle_big"
        android:backgroundTint="?attr/colorPrimary"
        android:contentDescription="@string/bullet"
        app:layout_constraintBottom_toBottomOf="@+id/RVcolorBarView"
        app:layout_constraintStart_toStartOf="parent"
        app:layout_constraintTop_toTopOf="@+id/RVcolorBarView" />

    <ImageView
        android:id="@+id/iconTimeLineImageView"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:layout_marginStart="12dp"
        android:layout_marginTop="8dp"
        android:background="@drawable/ic_car_repair"
        android:backgroundTint="@color/white"
        android:contentDescription="@string/bullet"
        app:layout_constraintBottom_toBottomOf="@+id/imageView"
        app:layout_constraintStart_toStartOf="@+id/imageView"
        app:layout_constraintTop_toTopOf="@+id/imageView"
        app:layout_constraintVertical_bias="0.285" />

    <TextView
        android:id="@+id/timeLineCostTypeNameTextView"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:padding="4dp"
        android:text="@string/costtype_name"
        android:textSize="24sp"
        app:layout_constraintBottom_toBottomOf="parent"
        app:layout_constraintStart_toEndOf="@+id/imageView"
        app:layout_constraintTop_toTopOf="parent" />

    <TextView
        android:id="@+id/timeLineCostAmountTextView"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:layout_marginEnd="16dp"
        android:padding="4dp"
        android:text="1200 zł"
        android:textSize="24sp"
        app:layout_constraintBottom_toBottomOf="parent"
        app:layout_constraintEnd_toEndOf="parent"
        app:layout_constraintTop_toTopOf="parent" />

    <TextView
        android:id="@+id/timeLineFullDateTextView"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:fontFamily="serif"
        android:paddingStart="4dp"
        android:text="@string/date"
        android:textSize="16sp"
        app:layout_constraintBottom_toBottomOf="parent"
        app:layout_constraintStart_toEndOf="@+id/imageView"
        app:layout_constraintTop_toBottomOf="@+id/timeLineCostTypeNameTextView" />

</androidx.constraintlayout.widget.ConstraintLayout>
```

Zauważmy że znów mamy `View` o tych samych parametrch co w poprzednim layoucie co sprawia wrażenie ciągłości przez wszystkie elementy. Mamy również dwa `ImageView` - pierwszy zastosowałem jako okrągłe tło o jednolitym kolorze dla ikony obecnej w drugim.

Przejdźmy do metody `onCreateViewHolder` - zwrócimy `ViewHolder` dla zadanego `viewType`. Ponieważ będziemy wykorzystywać dwa, musimy zdefiniować jeszcze `viewType`.

In [ ]:
override fun onCreateViewHolder(parent: ViewGroup, viewType: Int): RecyclerView.ViewHolder {
    return when (viewType) {
        CostListItem.TYPE_DATE ->
            DateViewHolder(LayoutInflater.from(parent.context).inflate(
                R.layout.date_item_timeline, parent, false
            ))
        else ->
            GeneralViewHolder(LayoutInflater.from(parent.context).inflate(
                R.layout.general_item_timeline, parent, false
            ))
    }
}

Aby zdefiniować `viewType` musimy nadpisać metodę `getItemViewType`

In [ ]:
override fun getItemViewType(position: Int): Int = itemList[position].type // zwraca typ elementu na liście

Zaimplementujmy metodę `getItemCount`

In [ ]:
override fun getItemCount(): Int = itemList.size

Pozostaje implementacja metody `onBindViewHolder` - w zależności od typu `ViewHolder` chcemy przekazać `CostListItem` jako `CostGeneralItem` zawierający instancję `Cost` lub `CostDateItem` zawierający nazwę miesiąca jako `String`.

In [ ]:
override fun onBindViewHolder(holder: RecyclerView.ViewHolder, position: Int) {
    when (holder.itemViewType) {
        CostListItem.TYPE_DATE -> (holder as DateViewHolder).bind(
            item = itemList[position] as CostDateItem
        )
        CostListItem.TYPE_GENERAL -> (holder as GeneralViewHolder).bind(
            item = itemList[position] as CostGeneralItem
        )
    }
}

Przejdźmy do `TimeLineFragment` i dodajmy nasz RecyclerView

In [ ]:
override fun onViewCreated(view: View, savedInstanceState: Bundle?) {
    super.onViewCreated(view, savedInstanceState)
    view.findViewById<RecyclerView>(R.id.timeLineRecyclerView).apply {
        layoutManager = LinearLayoutManager(context)
        adapter = TimeLineAdapter(context)
    }
}

Możemy przetestować aplikację

<img src="https://media4.giphy.com/media/S4iOJk9GSVcizP2tRw/giphy.gif?cid=790b76113b3cee84bee0e3484c2fbc03807cf26e022c376a&rid=giphy.gif&ct=g" width="150" />

### **Model danych**

Przygotujmy prosty model danych opisujący samochód

In [ ]:
data class Car (
    val name: String,
    val brand: String,
    val model: String,
    val yearOfProduction: Int,
    val costs: List<Cost>
)

Poza standardowymi polami umieścimy również listę wszystkich kosztów (do której nie będziemy dodawać kolejnych wpisów). Musimy wprowadzić niewielkie zmiany w `DataProvider` i wystawić liste wszystkich samochodów. Zmienimy stałą `generalCosts` na funkcję przyjmującą rozmiar listy.

In [ ]:
private fun  generalCosts(size: Int) = List(size) {
    Cost(
        CostType.values()[Random.nextInt(CostType.values().size)],
        LocalDate.of(2022, Random.nextInt(1,13), Random.nextInt(1,28)),
        Random.nextInt(5000)
    )
}

Zmodyfikujemy również nieco funkcję `getTimeLineList`

In [ ]:
fun getTimeLineList(costs: List<Cost>): List<CostListItem>{
    val groupedCostMap: Map<String, List<Cost>> = 
        costs.sortedBy { it.date.month }.groupBy { it.date.month.toString() }
        
    val list = mutableListOf<CostListItem>()
    
    for (date: String in groupedCostMap.keys) {
        val groupItems: List<Cost>? = groupedCostMap[date]
        groupItems?.sortedBy { it.date.dayOfMonth }?.forEach {
            list.add(CostGeneralItem(it))
        }
        list.add(CostDateItem(date))
    }
    return list
}

Chcemy mieć możliwość pokazania listy wszystkich kosztów dla każdego samochodu. Zmieńmy sygnaturę klasy `TimeLineAdapter` - będzie przyjmować listę wszystkich kosztów `List<Cost>`

In [ ]:
class TimeLineAdapter(private val context: Context, costList: List<Cost>) : RecyclerView.Adapter<RecyclerView.ViewHolder>() {

    private val itemList = DataProvider.getTimeLineList(costList).reversed()
    ...
}

Ostatnim elementem będzie wystawienie listy wszystkich samochodów przez obiekt `DataProvider`

In [ ]:
val cars = listOf(
    Car("Domowy", "Skoda", "Fabia", 2002, generalCosts(100)),
    Car("Służbowy", "BMW", "Coupe", 2015, generalCosts(50)),
    Car("Kolekcjonerski", "Fiat", "125p", 1985, generalCosts(10))
)

### **Menu rozwijane - Spinner**

Na `TimeLineFragment` będziemy zmieniać listę kosztów za pomocą `Spinner`. W pierwszym kroku dodajmy go do layoutu

```xml
<?xml version="1.0" encoding="utf-8"?>
<LinearLayout xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:tools="http://schemas.android.com/tools"
    android:layout_width="match_parent"
    android:layout_height="match_parent"
    android:orientation="vertical"
    tools:context=".fragments.TimeLineFragment">

    <Spinner
        android:id="@+id/cars_spinner"
        android:layout_width="match_parent"
        android:drawSelectorOnTop="true"
        android:background="@drawable/spinner_bg"
        style="@style/Widget.AppCompat.Spinner"
        android:layout_marginTop="20dp"
        android:layout_marginStart="10dp"
        android:layout_marginEnd="10dp"
        android:layout_height="wrap_content" />

    <androidx.recyclerview.widget.RecyclerView
        android:id="@+id/timeLineRecyclerView"
        android:layout_width="match_parent"
        android:layout_height="0dp"
        android:layout_weight="1"
        android:layout_marginTop="12dp" />

</LinearLayout>
```

W pliku `spinner_bg` w katalogu `drawable` definiujemy tło naszego spinnera

```xml
<?xml version="1.0" encoding="utf-8"?>
<selector xmlns:android="http://schemas.android.com/apk/res/android"
    android:exitFadeDuration="@android:integer/config_mediumAnimTime">
    <item>
        <inset android:insetLeft="-1dp" android:insetRight="-1dp">
            <shape android:shape="rectangle">
                <stroke android:width="1dp"
                    android:color="@color/black" />
                <solid android:color="?attr/colorPrimary" />
                <corners android:radius="100dp"/>
            </shape>
        </inset>
    </item>
</selector>
```

Kolejnym krokiem będzie utworzenie layoutu pojedynczego elementu listy `Spinner` - będzie to pole `TextView`. W katalogu `layout` tworzymy nowy plik `spinner_layout`

```xml
<?xml version="1.0" encoding="utf-8"?>
<TextView xmlns:android="http://schemas.android.com/apk/res/android"
    android:id="@+id/spinner_layout"
    android:layout_width="match_parent"
    android:layout_height="wrap_content"
    android:ellipsize="marquee"
    android:fontFamily="sans-serif"
    android:gravity="center"
    android:textColor="?attr/colorOnPrimary"
    android:background="@drawable/spinner_bg"
    android:singleLine="true"
    android:text="CAR"
    android:padding="10dp"
    android:textSize="35sp" />
```

Pozostało utworzyć `Spinner` w metodzie `onViewVCreated` klasy `TimeLineFragment`. Będziemy przeładowywać cały adapter w `RecyclerView`, więc tworzymy odpowiednią zmienną

In [ ]:
class TimeLineFragment : Fragment() {

    override fun onCreateView(
        inflater: LayoutInflater, container: ViewGroup?,
        savedInstanceState: Bundle?
    ): View? {
        // Inflate the layout for this fragment
        return inflater.inflate(R.layout.fragment_time_line, container, false)
    }

    override fun onViewCreated(view: View, savedInstanceState: Bundle?) {
        super.onViewCreated(view, savedInstanceState)
        val recycler = view.findViewById<RecyclerView>(R.id.timeLineRecyclerView).apply {
            layoutManager = LinearLayoutManager(context)
            adapter = TimeLineAdapter(context, DataProvider.cars[0].costs)
        }
    }
}

Przejdźmy do obsłużenia `Spinner`, tutaj również będziemy potrzebować odpowiedni adapter. Adapterem który wykorzystamy będzie `ArrayAdapter` przyjmuje on trzy argumenty
- `context`
- `layout`
- listę danych
Jako dane podajemy listę naszych nazw samochodów

In [ ]:
view.findViewById<Spinner>(R.id.cars_spinner).apply {
    adapter = ArrayAdapter(context, R.layout.spinner_layout, DataProvider.cars.map { it.name })
}

Następnie chcemy podmienić adapter i odświeżyć `RecyclerView` po wybraniu opcji na `Spinner` - w tym calu implementujemy `AdapterView.OnItemSelectedListener`. Ten interfejs zawiera dwie metody
- `onItemSelected`
- `onNothingSelected`

In [ ]:
view.findViewById<Spinner>(R.id.cars_spinner).apply {
    adapter = ArrayAdapter(context, R.layout.spinner_layout, DataProvider.cars.map { it.name })
    onItemSelectedListener = object : AdapterView.OnItemSelectedListener{
        override fun onItemSelected(parent: AdapterView<*>?,
                                    view: View?, position: Int,
                                    id: Long) {}
        override fun onNothingSelected(parent: AdapterView<*>?) {}
}

Nas interesuje tylko pierwsza, `onNothingSelected` pozostawimy pustą. Wywołamy metodę `swapAdapter` klasy `RecyclerView` - przyjmuje ona dwa parametry
- adapter - nowy adapter, który zastąpi poprzedni
- `removeAndRecycleExistingViews: Boolean` - jeśli ustawimy jako `true` wszystkie utworzone do tej pory obiekty `View` zostaną usunięte

In [ ]:
override fun onViewCreated(view: View, savedInstanceState: Bundle?) {
    super.onViewCreated(view, savedInstanceState)
    val recycler = view.findViewById<RecyclerView>(R.id.timeLineRecyclerView).apply {
        layoutManager = LinearLayoutManager(context)
        adapter = TimeLineAdapter(context, DataProvider.cars[0].costs)
    }
    
    view.findViewById<Spinner>(R.id.cars_spinner).apply {
        adapter = ArrayAdapter(context, 
                               R.layout.spinner_layout,
                               DataProvider.cars.map { it.name })
        onItemSelectedListener = object : AdapterView.OnItemSelectedListener{
            override fun onItemSelected(parent: AdapterView<*>?,
                                        view: View?, position: Int,
                                        id: Long) {
                recycler.swapAdapter(TimeLineAdapter(context, DataProvider.cars[position].costs), true)
            }

            override fun onNothingSelected(parent: AdapterView<*>?) {}
        }
    }
}

Możemy przetestować aplikację

<img src="https://media2.giphy.com/media/Qo4yNLd6gn7090SWRl/giphy.gif?cid=790b7611251b7107f210b555df82ce9f979a76103dae2791&rid=giphy.gif&ct=g" width="150" />

### **CalculatorsFragment**

Dodamy możliwość skorzystania z trzech kalkulatorów pozwalających obliczyć podstawowe parametry na podstawie informacji o paliwie, spalaniu, cenie i dystansie. Kalkulatory będą współdzielić layout, zmianę będziemy dokonywać za pomocą `Spinner`.

Będziemy posługiwać się liczbami zmiennoprzecinkowymi, więc dodajmy zaokrąglenie do dwóch miejsc. W pliku `Formatter` dodajemy

In [ ]:
val decimalFormat = DecimalFormat("#.##")

Przygotujmy layout z formularzem oraz spinnerem

```xml
<?xml version="1.0" encoding="utf-8"?>
<LinearLayout xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:tools="http://schemas.android.com/tools"
    android:layout_width="match_parent"
    android:layout_height="match_parent"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    android:orientation="vertical"
    tools:context=".fragments.TimeLineFragment">

    <Spinner
        android:id="@+id/calculators_spinner"
        style="@style/Widget.AppCompat.Spinner"
        android:layout_width="match_parent"
        android:layout_height="wrap_content"
        android:layout_marginStart="10dp"
        android:layout_marginTop="20dp"
        android:layout_marginEnd="10dp"
        android:background="@drawable/spinner_bg"
        android:drawSelectorOnTop="true" />

    <androidx.cardview.widget.CardView
        android:layout_width="match_parent"
        android:layout_height="wrap_content"
        android:layout_marginStart="10dp"
        android:layout_marginTop="20dp"
        android:layout_marginEnd="10dp"
        android:backgroundTint="?attr/colorPrimary"
        android:outlineProvider="background"
        app:cardCornerRadius="35dp">

        <LinearLayout
            android:layout_width="match_parent"
            android:layout_height="match_parent"
            android:layout_marginTop="20dp"
            android:orientation="vertical">

            <com.google.android.material.textfield.TextInputLayout
                android:id="@+id/editInputLayout1"
                style="@style/Widget.MaterialComponents.TextInputLayout.OutlinedBox"
                android:layout_width="match_parent"
                android:layout_height="wrap_content"
                android:hint="@string/odleg_o"
                app:boxStrokeColor="#fff"
                android:theme="@style/editText"
                app:hintTextColor="?attr/colorOnPrimary"
                android:textColorHint="#fff">

                <com.google.android.material.textfield.TextInputEditText
                    android:id="@+id/editText1"
                    android:layout_width="match_parent"
                    android:layout_height="wrap_content"
                    android:layout_marginStart="50dp"
                    android:layout_marginTop="20dp"
                    android:layout_marginEnd="50dp"
                    android:gravity="center"
                    android:textColor="?attr/colorOnPrimary"
                    android:textSize="18sp"
                    android:textColorHighlight="?attr/colorOnPrimary"
                    android:inputType="number" />
            </com.google.android.material.textfield.TextInputLayout>

            <com.google.android.material.textfield.TextInputLayout
                android:id="@+id/editInputLayout2"
                style="@style/Widget.MaterialComponents.TextInputLayout.OutlinedBox"
                android:layout_width="match_parent"
                android:layout_height="wrap_content"
                android:hint="@string/cena_za_litr"
                app:boxStrokeColor="?attr/colorOnPrimary"
                android:theme="@style/editText"
                app:hintTextColor="?attr/colorOnPrimary"
                android:textColorHint="#fff">

                <com.google.android.material.textfield.TextInputEditText
                    android:id="@+id/editText2"
                    android:layout_width="match_parent"
                    android:layout_height="wrap_content"
                    android:layout_marginStart="50dp"
                    android:layout_marginTop="20dp"
                    android:layout_marginEnd="50dp"
                    android:gravity="center"
                    android:textColor="?attr/colorOnPrimary"
                    android:textSize="18sp"
                    android:textColorHighlight="@color/white"
                    android:inputType="numberDecimal" />
            </com.google.android.material.textfield.TextInputLayout>

            <com.google.android.material.textfield.TextInputLayout
                android:id="@+id/editInputLayout3"
                style="@style/Widget.MaterialComponents.TextInputLayout.OutlinedBox"
                android:layout_width="match_parent"
                android:layout_height="wrap_content"
                android:hint="@string/spalanie"
                app:boxStrokeColor="?attr/colorOnPrimary"
                android:theme="@style/editText"
                app:hintTextColor="?attr/colorOnPrimary"
                android:textColorHint="?attr/colorOnPrimary">

                <com.google.android.material.textfield.TextInputEditText
                    android:id="@+id/editText3"
                    android:layout_width="match_parent"
                    android:layout_height="wrap_content"
                    android:layout_marginStart="50dp"
                    android:layout_marginTop="20dp"
                    android:layout_marginEnd="50dp"
                    android:gravity="center"
                    android:textColor="?attr/colorOnPrimary"
                    android:textSize="18sp"
                    android:textColorHighlight="?attr/colorOnPrimary"
                    android:inputType="numberDecimal" />
            </com.google.android.material.textfield.TextInputLayout>

            <LinearLayout
                android:layout_width="match_parent"
                android:layout_height="wrap_content"
                android:layout_marginTop="40dp"
                android:layout_marginStart="15dp"
                android:orientation="horizontal">

                <com.google.android.material.textview.MaterialTextView
                    android:id="@+id/textViewMainTitle"
                    android:layout_width="wrap_content"
                    android:layout_height="wrap_content"
                    android:textColor="?attr/colorOnPrimary"
                    android:text="MainTitle"
                    android:textStyle="bold"
                    android:textSize="24sp" />

                <com.google.android.material.textview.MaterialTextView
                    android:id="@+id/textViewMainValue"
                    android:layout_width="0dp"
                    android:layout_weight="1"
                    android:textAlignment="viewEnd"
                    android:layout_height="wrap_content"
                    android:textColor="?attr/colorOnPrimary"
                    android:textStyle="bold"
                    android:text=""
                    android:layout_marginEnd="20dp"
                    android:textSize="24sp" />

            </LinearLayout>
            <com.google.android.material.textview.MaterialTextView
                android:id="@+id/textViewBottomValue"
                android:layout_width="match_parent"
                android:textAlignment="viewEnd"
                android:layout_height="wrap_content"
                android:textColor="?attr/colorOnPrimary"
                android:text=""
                android:layout_marginEnd="20dp"
                android:layout_marginStart="20dp"
                android:textSize="18sp"
                android:layout_marginBottom="20dp"/>
        </LinearLayout>
    </androidx.cardview.widget.CardView>

    <com.google.android.material.button.MaterialButton
        android:id="@+id/calculateButton"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:backgroundTint="@android:color/holo_red_dark"
        android:layout_gravity="right"
        android:layout_marginEnd="35dp"
        android:text="Oblicz"/>

</LinearLayout>
```

Aby wykorzystać inną niż domyślna kolorystyka pól `EditText`, dodajmy do `themes.xml` własny styl

```xml
<style name="editText" parent="Widget.MaterialComponents.TextInputLayout.OutlinedBox.Dense">
    <item name="boxStrokeColor">#fff</item>
    <item name="boxStrokeWidth">2dp</item>
</style>
```

Zwróćmy uwagę że pole `EditText` jest zastąpione przez wersję dodstępną w `Materials Design` - możemy zwrócić uwagę że w pliku `gradle` w bloku `dependencies` domyślnie mamy tą bibliotekę już dodaną

```kotlin
implementation 'com.google.android.material:material:1.6.1'
```

Zwykłe pole `EditText` jest zastąpione przez dwa pole
- `TextInputLayout` - zawierający podpowiedź i kilka elementów graficznych
- `TextInputEditText` - sam `EditText`

Kolejnym elementem jest modyfikacja wejścia w tych polach
- `android:inputType="number"` - liczby całkowite - zostanie wyświetlona tylko klawiatura numeryczna
- `android:inputType="numberDecimal"` - liczby zmiennoprzecinkowe

Zamieszczonym przyciskiem będziemy wykonywać obliczenie.

Przejdźmy do klasy `CalculatorsFragment` i metody `onViewCreated` - dodajmy `Spinner`

In [ ]:
view.findViewById<Spinner>(R.id.calculators_spinner).apply {

    val titles = listOf("Koszt podróży", "Odległość", "Wymagane paliwo")
    adapter = ArrayAdapter(context, R.layout.spinner_layout, titles)
}

Wykorzystujemy ten sam layout co w poprzednim `Spinner`. Dodajmy implementację `onItemSelected`

In [ ]:
onItemSelectedListener = object : AdapterView.OnItemSelectedListener {
    override fun onItemSelected(
        parent: AdapterView<*>?,
        v: View?, position: Int,
        id: Long
    ) {}

W pierwszym kroku ustawimy tekst pola `textViewMainTitle` jako nazwę kalkulatora oraz w zależności od wybranego kalkulatora zmienimy podpowiedź jednego pola `TextInputLayout`

In [ ]:
view.findViewById<TextView>(R.id.textViewMainTitle).text = titles[position]
view.findViewById<TextInputLayout>(R.id.editInputLayout1).hint =
    if (position == 1) "Paliwo [l]" else "Odległość [km]"

Kolejnym krokiem będzie obsługa `onCLick` przycisku

In [ ]:
view.findViewById<MaterialButton>(R.id.calculateButton).setOnClickListener {}

Rozpocznijmy od wyjęcia wszystkich danych z pól `EditText`

In [ ]:
val input = Triple(
    view.findViewById<EditText>(R.id.editText1).text.toString(),
    view.findViewById<EditText>(R.id.editText2).text.toString(),
    view.findViewById<EditText>(R.id.editText3).text.toString()
)

Na podstawie tych danych (w zależności od wybranego kalkulatora) chcemy przeprowadzić odpowiednie obliczenia, zróbmy to w metodzie `calculators`

In [ ]:
private fun calculators(
    position: Int,
    values: Triple<String, String, String>
): Pair<String, String> {
    return when (position) {
        0 -> {
            val stringBottom = decimalFormat.format((values.third.toDouble() / 100) * 
                    values.first.toDouble()).toString()
            val stringMain = decimalFormat.format(stringBottom.toDouble() * 
                    values.second.toDouble()).toString()
            Pair("$stringMain zł", "$stringBottom l")
        }
        1 -> {
            val stringBottom = decimalFormat.format((values.first.toDouble()) * 
                    values.second.toDouble()).toString()
            val stringMain = decimalFormat.format((100 * values.first.toDouble()) / 
                    values.third.toDouble()).toString()
            Pair("$stringMain km", "$stringBottom zł")
        }
        2 -> {
            val stringMain = decimalFormat.format((values.first.toDouble() * 
                    values.third.toDouble()) / 100).toString()
            val stringBottom = decimalFormat.format(stringMain.toDouble() *
                    values.second.toDouble()).toString()
            Pair("$stringMain l", "$stringBottom zł")
        }
        else -> Pair("", "")
    }
}

Każdy kalkulator oblicza dwie wielkości, więc zwracam je jako `Pair<String, String>`. W meotdzie `onClick` sprawdzamy czy to co dastajemy z pól `EditText` nie jest puste i wywołujemy powyższą metodę - wyniki umieszczamy na odpowiednich polach `TextView`

In [ ]:
if (input.first.isNotEmpty() && input.second.isNotEmpty() && input.third.isNotEmpty()) {
    val values: Pair<String, String> = calculators(position, input)
    view.findViewById<TextView>(R.id.textViewMainValue).text = values.first
    view.findViewById<TextView>(R.id.textViewBottomValue).text = values.second
}

Pełna metoda `onViewCreated`:

In [ ]:
override fun onViewCreated(view: View, savedInstanceState: Bundle?) {
    super.onViewCreated(view, savedInstanceState)

    view.findViewById<Spinner>(R.id.calculators_spinner).apply {

        val titles = listOf("Koszt podróży", "Odległość", "Wymagane paliwo")
        adapter = ArrayAdapter(context, R.layout.spinner_layout, titles)

        onItemSelectedListener = object : AdapterView.OnItemSelectedListener {
            override fun onItemSelected(
                parent: AdapterView<*>?,
                v: View?, position: Int,
                id: Long
            ) {
                view.findViewById<TextView>(R.id.textViewMainTitle).text = titles[position]
                view.findViewById<TextInputLayout>(R.id.editInputLayout1).hint =
                    if (position == 1) "Paliwo [l]" else "Odległość [km]"

                view.findViewById<MaterialButton>(R.id.calculateButton).setOnClickListener {
                    val input = Triple(
                        view.findViewById<EditText>(R.id.editText1).text.toString(),
                        view.findViewById<EditText>(R.id.editText2).text.toString(),
                        view.findViewById<EditText>(R.id.editText3).text.toString()
                    )

                    if (input.first.isNotEmpty() 
                        && input.second.isNotEmpty() 
                        && input.third.isNotEmpty()) {
                        val values: Pair<String, String> = calculators(position, input)
                        view.findViewById<TextView>(R.id.textViewMainValue).text = values.first
                        view.findViewById<TextView>(R.id.textViewBottomValue).text = values.second
                    }
                }
            }

            override fun onNothingSelected(parent: AdapterView<*>?) {}
        }
    }
}

Możemy przetestować aplikację

<img src="https://media1.giphy.com/media/6Ee7Ygm11rzDTIbGYW/giphy.gif?cid=790b761123f95e9b3caef44cdf2de4af63e5e1962bbee76a&rid=giphy.gif&ct=g" width="150" />

Zmieńmy nieco wygląd aplikacji - usuńmy plik `themes.xml(night)`. W pliku `colors.xml` zdefiniujmy kilka kolorów

```xml
<?xml version="1.0" encoding="utf-8"?>
<resources>
    <color name="purple_200">#FFBB86FC</color>
    <color name="purple_500">#FF6200EE</color>
    <color name="purple_700">#FF3700B3</color>
    <color name="teal_200">#FF03DAC5</color>
    <color name="teal_700">#FF018786</color>
    <color name="black">#FF000000</color>
    <color name="white">#FFFFFFFF</color>
    <color name="mtrl_textinput_default_box_stroke_color">#fff</color>
    <color name="green_500">#FF1EB980</color>
    <color name="dark_blue_900">#FF26282F</color>
    <color name="dark_blue_500">#FF004940</color>

</resources>
```

Następnie zmodyfikujmy plik `themes.xml`

```xml
<resources xmlns:tools="http://schemas.android.com/tools">
    <!-- Base application theme. -->
    <style name="Theme.CarsyAppKotlin" parent="Theme.MaterialComponents.DayNight.NoActionBar">
        <!-- Primary brand color. -->
        <item name="colorPrimary">@color/dark_blue_500</item>
        <item name="colorPrimaryVariant">@color/dark_blue_900</item>
        <item name="colorOnPrimary">@color/teal_700</item>
        <!-- Secondary brand color. -->
        <item name="colorSecondary">@color/teal_700</item>
        <item name="colorSecondaryVariant">@color/teal_200</item>
        <item name="colorOnSecondary">@color/white</item>
        <!-- Status bar color. -->
        <item name="android:statusBarColor" tools:targetApi="l">?attr/colorPrimaryVariant</item>
        <!-- Customize your theme here. -->
    </style>

    <style name="editText" parent="Widget.MaterialComponents.TextInputLayout.OutlinedBox.Dense">
        <item name="boxStrokeColor">@color/teal_200</item>
        <item name="boxStrokeWidth">2dp</item>
    </style>
</resources>
```

Dodajmy osobny layout dla menu rozwijanego w `Spinner` (`spinner_dropdown_layoutx.xml`)

```xml
<?xml version="1.0" encoding="utf-8"?>
<TextView xmlns:android="http://schemas.android.com/apk/res/android"
    android:id="@+id/spinner_layout"
    android:layout_width="match_parent"
    android:layout_height="wrap_content"
    android:ellipsize="marquee"
    android:fontFamily="sans-serif"
    android:gravity="center"
    android:textColor="@color/teal_200"
    android:background="@drawable/spinner_dropdown_bg"
    android:singleLine="true"
    android:text="CAR"
    android:padding="10dp"
    android:textSize="35sp" />
```

W klasach `TimeLineFragment` i `CalculatorsFragment` wykorzystajmy utworzony layout

In [ ]:
...
view.findViewById<Spinner>(R.id.cars_spinner).apply {
            adapter = ArrayAdapter(context, R.layout.spinner_layout, DataProvider.cars.map { it.name })
                .apply { setDropDownViewResource(R.layout.spinner_dropdown_layout) }
...

Przejdźmy do layoutu głównej aktywności i zmodyfikujmy kolorystykę `BottomNavigationView`

```xml
<com.google.android.material.bottomnavigation.BottomNavigationView
    android:id="@+id/bottom_nav_view"
    android:layout_width="match_parent"
    android:layout_height="wrap_content"
    android:background="@color/dark_blue_900"
    app:itemIconTint="@drawable/bottom_navigation_highlight"
    app:itemTextColor="@drawable/bottom_navigation_highlight"
    app:menu="@menu/bottom_menu" />
```

Pozbywamy się w ten sposób domyślnych ustawień dla podświetlenie zaznaczonej ikony oraz tekstu, więc zdefiniujmy własny w pliku `bottom_navigation_highlight.xml` (katalog `drawable`)

```xml
<?xml version="1.0" encoding="utf-8"?>
<selector xmlns:android="http://schemas.android.com/apk/res/android">
    <item android:state_checked="true" android:color="@color/teal_200" />
    <item android:state_checked="false" android:color="@color/teal_700"/>
</selector>
```

W plikach `date_item_timeline.xml` i `general_item_timeline.xml` zmieniamy kolor tła na

```xml
android:background="@color/dark_blue_900"
```

oraz kolor podpowiedzi na

```xml
android:backgroundTint="@color/teal_200"
```

Modyfikujemy layout `CalculatorsFragment`

```xml
<?xml version="1.0" encoding="utf-8"?>
<LinearLayout xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:tools="http://schemas.android.com/tools"
    android:layout_width="match_parent"
    android:layout_height="match_parent"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    android:orientation="vertical"
    android:background="@color/dark_blue_900"
    tools:context=".fragments.TimeLineFragment">

    <Spinner
        android:id="@+id/calculators_spinner"
        style="@style/Widget.AppCompat.Spinner"
        android:layout_width="match_parent"
        android:layout_height="wrap_content"
        android:layout_marginStart="10dp"
        android:popupBackground="@android:color/transparent"
        android:layout_marginTop="20dp"
        android:dropDownVerticalOffset="64dp"
        android:popupElevation="3dp"
        android:layout_marginEnd="10dp"
        android:background="@drawable/spinner_bg"
        android:drawSelectorOnTop="true" />

    <androidx.cardview.widget.CardView
        android:layout_width="match_parent"
        android:layout_height="wrap_content"
        android:layout_marginStart="10dp"
        android:layout_marginTop="20dp"
        android:layout_marginEnd="10dp"
        android:backgroundTint="?attr/colorPrimary"
        android:outlineProvider="background"
        app:cardCornerRadius="35dp">

        <LinearLayout
            android:layout_width="match_parent"
            android:layout_height="match_parent"
            android:layout_marginTop="20dp"
            android:orientation="vertical">

            <com.google.android.material.textfield.TextInputLayout
                android:id="@+id/editInputLayout1"
                style="@style/Widget.MaterialComponents.TextInputLayout.OutlinedBox"
                android:layout_width="match_parent"
                android:layout_height="wrap_content"
                android:hint="@string/odleg_o"
                app:boxStrokeColor="@color/teal_200"
                android:theme="@style/editText"
                app:hintTextColor="@color/teal_200"
                android:textColorHint="@color/teal_200">

                <com.google.android.material.textfield.TextInputEditText
                    android:id="@+id/editText1"
                    android:layout_width="match_parent"
                    android:layout_height="wrap_content"
                    android:layout_marginStart="50dp"
                    android:layout_marginTop="20dp"
                    android:layout_marginEnd="50dp"
                    android:gravity="center"
                    android:textColor="@color/teal_200"
                    android:textSize="18sp"
                    android:textColorHighlight="@color/teal_200"
                    android:inputType="number" />
            </com.google.android.material.textfield.TextInputLayout>

            <com.google.android.material.textfield.TextInputLayout
                android:id="@+id/editInputLayout2"
                style="@style/Widget.MaterialComponents.TextInputLayout.OutlinedBox"
                android:layout_width="match_parent"
                android:layout_height="wrap_content"
                android:hint="@string/cena_za_litr"
                app:boxStrokeColor="@color/teal_200"
                android:theme="@style/editText"
                app:hintTextColor="@color/teal_200"
                android:textColorHint="@color/teal_200">

                <com.google.android.material.textfield.TextInputEditText
                    android:id="@+id/editText2"
                    android:layout_width="match_parent"
                    android:layout_height="wrap_content"
                    android:layout_marginStart="50dp"
                    android:layout_marginTop="20dp"
                    android:layout_marginEnd="50dp"
                    android:gravity="center"
                    android:textColor="@color/teal_200"
                    android:textSize="18sp"
                    android:textColorHighlight="@color/teal_200"
                    android:inputType="numberDecimal" />
            </com.google.android.material.textfield.TextInputLayout>

            <com.google.android.material.textfield.TextInputLayout
                android:id="@+id/editInputLayout3"
                style="@style/Widget.MaterialComponents.TextInputLayout.OutlinedBox"
                android:layout_width="match_parent"
                android:layout_height="wrap_content"
                android:hint="@string/spalanie"
                app:boxStrokeColor="@color/teal_200"
                android:theme="@style/editText"
                app:hintTextColor="@color/teal_200"
                android:textColorHint="@color/teal_200">

                <com.google.android.material.textfield.TextInputEditText
                    android:id="@+id/editText3"
                    android:layout_width="match_parent"
                    android:layout_height="wrap_content"
                    android:layout_marginStart="50dp"
                    android:layout_marginTop="20dp"
                    android:layout_marginEnd="50dp"
                    android:gravity="center"
                    android:textColor="@color/teal_200"
                    android:textSize="18sp"
                    android:textColorHighlight="@color/teal_200"
                    android:inputType="numberDecimal" />
            </com.google.android.material.textfield.TextInputLayout>

            <LinearLayout
                android:layout_width="match_parent"
                android:layout_height="wrap_content"
                android:layout_marginTop="40dp"
                android:layout_marginStart="15dp"
                android:orientation="horizontal">

                <com.google.android.material.textview.MaterialTextView
                    android:id="@+id/textViewMainTitle"
                    android:layout_width="wrap_content"
                    android:layout_height="wrap_content"
                    android:textColor="@color/teal_200"
                    android:text="MainTitle"
                    android:textStyle="bold"
                    android:textSize="24sp" />

                <com.google.android.material.textview.MaterialTextView
                    android:id="@+id/textViewMainValue"
                    android:layout_width="0dp"
                    android:layout_weight="1"
                    android:textAlignment="viewEnd"
                    android:layout_height="wrap_content"
                    android:textColor="?attr/colorOnPrimary"
                    android:textStyle="bold"
                    android:text=""
                    android:layout_marginEnd="20dp"
                    android:textSize="24sp" />

            </LinearLayout>
            <com.google.android.material.textview.MaterialTextView
                android:id="@+id/textViewBottomValue"
                android:layout_width="match_parent"
                android:textAlignment="viewEnd"
                android:layout_height="wrap_content"
                android:textColor="@color/teal_200"
                android:text=""
                android:layout_marginEnd="20dp"
                android:layout_marginStart="20dp"
                android:textSize="18sp"
                android:layout_marginBottom="20dp"/>
        </LinearLayout>
    </androidx.cardview.widget.CardView>

    <com.google.android.material.button.MaterialButton
        android:id="@+id/calculateButton"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:backgroundTint="@android:color/holo_red_dark"
        android:textColor="@color/white"
        android:layout_gravity="right"
        android:textSize="18sp"
        android:layout_marginEnd="35dp"
        android:text="Oblicz"/>

</LinearLayout>
```

Również modyfikujemy layout `TimeLineFragment`

```xml
<?xml version="1.0" encoding="utf-8"?>
<LinearLayout xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:tools="http://schemas.android.com/tools"
    android:layout_width="match_parent"
    android:layout_height="match_parent"
    android:orientation="vertical"
    android:background="@color/dark_blue_900"
    tools:context=".fragments.TimeLineFragment">

    <Spinner
        android:id="@+id/cars_spinner"
        android:layout_width="match_parent"
        android:drawSelectorOnTop="true"
        android:popupBackground="@android:color/transparent"
        android:background="@drawable/spinner_bg"
        style="@style/Widget.AppCompat.Spinner"
        android:layout_marginTop="20dp"
        android:popupElevation="3dp"
        android:layout_marginStart="10dp"
        android:dropDownVerticalOffset="64dp"
        android:layout_marginEnd="10dp"
        android:layout_height="wrap_content" />

    <androidx.recyclerview.widget.RecyclerView
        android:id="@+id/timeLineRecyclerView"
        android:layout_width="match_parent"
        android:layout_height="0dp"
        android:layout_weight="1"
        android:layout_marginTop="12dp" />

</LinearLayout>
```

Po zmianach aplikacją powinna wyglądać następująco

<img src="https://media1.giphy.com/media/27zha8NAaWcBwWCJ6Q/giphy.gif?cid=790b7611b21116465fd1c582d6ad6ce899d6a4cb9e89cd4b&rid=giphy.gif&ct=g" width="150" />

### **OverviewFragment**

Do fragmentu zawierającego przegląd dodamy dwa `RecyclerView` zawierające podsumowanie wydatków oraz przegląd wszystkich samochodów. Jak również informację o całkowitych poniesionych kosztach. Rozpocznijmy od layoutu

Ponieważ nie wszystko może zostać zmieszczone na jednym ekranie naszym głównym elementem będzie `ScrollView`

```xml
<ScrollView  xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:tools="http://schemas.android.com/tools"
    android:layout_width="match_parent"
    android:background="@color/dark_blue_900"
    android:layout_height="match_parent">
```

Pozostałe elementy umieścimy w `LinearLayout`

```xml
<LinearLayout
    android:layout_width="match_parent"
    android:layout_height="wrap_content"
    android:orientation="vertical"
    android:background="@color/dark_blue_900"
    tools:context=".fragments.OverviewFragment">
```

Dodajmy dwa `RecyclerView` z polami `TextView`

```xml
    <TextView
        android:layout_width="match_parent"
        android:layout_height="wrap_content"
        android:textSize="24sp"
        android:text="@string/podsumowanie"
        android:gravity="center"
        android:textColor="@color/teal_200"
        android:textStyle="bold"
        android:layout_marginTop="24dp"
        />

    <androidx.recyclerview.widget.RecyclerView
        android:id="@+id/overviewCostsRecyclerView"
        android:layout_width="match_parent"
        android:layout_height="wrap_content"
        android:layout_marginTop="12dp" />

    <TextView
        android:layout_width="match_parent"
        android:layout_height="wrap_content"
        android:textSize="24sp"
        android:text="@string/gara"
        android:gravity="center"
        android:textColor="@color/teal_200"
        android:textStyle="bold"
        android:layout_marginTop="24dp"
        />

    <androidx.recyclerview.widget.RecyclerView
        android:id="@+id/overviewCarsRecyclerView"
        android:layout_width="match_parent"
        android:layout_height="wrap_content"
        android:layout_marginTop="12dp" />
```

Na końcu umieścimy `CardView` zawierający pola przeznaczone do przedstawienie całkowitych kosztów

```xml
    <androidx.cardview.widget.CardView
        xmlns:app="http://schemas.android.com/apk/res-auto"
        android:layout_width="match_parent"
        android:layout_height="wrap_content"
        android:layout_marginTop="12dp"
        android:layout_marginStart="40dp"
        android:layout_marginEnd="40dp"
        app:cardCornerRadius="45dp">

        <androidx.constraintlayout.widget.ConstraintLayout
            android:layout_width="match_parent"
            android:layout_height="wrap_content"
            android:layout_marginStart="4dp"
            android:layout_marginEnd="4dp"
            android:background="@color/dark_blue_500">

            <TextView
                android:id="@+id/overviewCarName"
                android:layout_width="match_parent"
                android:layout_height="wrap_content"
                android:background="@color/green_500"
                android:gravity="center"
                android:padding="4dp"
                android:text="@string/koszty_ca_kowite"
                android:textColor="@color/black"
                android:textSize="24sp"
                app:layout_constraintStart_toStartOf="parent"
                app:layout_constraintTop_toTopOf="parent" />

            <TextView
                android:layout_width="match_parent"
                android:layout_height="wrap_content"
                android:padding="4dp"
                android:layout_marginStart="24dp"
                android:text="@string/tankowanie"
                android:textColor="@color/teal_200"
                android:textSize="18sp"
                app:layout_constraintStart_toStartOf="parent"
                app:layout_constraintTop_toBottomOf="@+id/overviewCarName" />

            <TextView
                android:id="@+id/overviewFueling"
                android:layout_width="match_parent"
                android:layout_height="wrap_content"
                android:padding="4dp"
                android:gravity="end"
                android:layout_marginEnd="24dp"
                android:text="1 000 000.00 zł"
                android:textColor="@color/teal_200"
                android:textSize="18sp"
                app:layout_constraintStart_toStartOf="parent"
                app:layout_constraintTop_toBottomOf="@+id/overviewCarName" />

            <TextView
                android:layout_width="match_parent"
                android:layout_height="wrap_content"
                android:padding="4dp"
                android:layout_marginStart="24dp"
                android:text="@string/serwis"
                android:textColor="@color/teal_200"
                android:textSize="18sp"
                app:layout_constraintStart_toStartOf="parent"
                app:layout_constraintTop_toBottomOf="@+id/overviewFueling" />

            <TextView
                android:id="@+id/overviewService"
                android:layout_width="match_parent"
                android:layout_height="wrap_content"
                android:padding="4dp"
                android:gravity="end"
                android:layout_marginEnd="24dp"
                android:text="1 000 000.00 zł"
                android:textColor="@color/teal_200"
                android:textSize="18sp"
                app:layout_constraintStart_toStartOf="parent"
                app:layout_constraintTop_toBottomOf="@+id/overviewFueling" />

            <TextView
                android:layout_width="match_parent"
                android:layout_height="wrap_content"
                android:padding="4dp"
                android:layout_marginStart="24dp"
                android:text="@string/parkowanie"
                android:textColor="@color/teal_200"
                android:textSize="18sp"
                app:layout_constraintStart_toStartOf="parent"
                app:layout_constraintTop_toBottomOf="@+id/overviewService" />

            <TextView
                android:id="@+id/overviewParking"
                android:layout_width="match_parent"
                android:layout_height="wrap_content"
                android:padding="4dp"
                android:gravity="end"
                android:layout_marginEnd="24dp"
                android:text="1 000 000.00 zł"
                android:textColor="@color/teal_200"
                android:textSize="18sp"
                app:layout_constraintStart_toStartOf="parent"
                app:layout_constraintTop_toBottomOf="@+id/overviewService" />

            <TextView
                android:layout_width="match_parent"
                android:layout_height="wrap_content"
                android:padding="4dp"
                android:layout_marginStart="24dp"
                android:text="@string/ubezpieczenie"
                android:textColor="@color/teal_200"
                android:textSize="18sp"
                app:layout_constraintStart_toStartOf="parent"
                app:layout_constraintTop_toBottomOf="@+id/overviewParking" />

            <TextView
                android:id="@+id/overviewInsurance"
                android:layout_width="match_parent"
                android:layout_height="wrap_content"
                android:padding="4dp"
                android:gravity="end"
                android:layout_marginEnd="24dp"
                android:text="1 000 000.00 zł"
                android:textColor="@color/teal_200"
                android:textSize="18sp"
                app:layout_constraintStart_toStartOf="parent"
                app:layout_constraintTop_toBottomOf="@+id/overviewParking" />

            <TextView
                android:layout_width="match_parent"
                android:layout_height="wrap_content"
                android:padding="4dp"
                android:layout_marginStart="24dp"
                android:text="@string/mandat"
                android:textColor="@color/teal_200"
                android:textSize="18sp"
                app:layout_constraintStart_toStartOf="parent"
                app:layout_constraintTop_toBottomOf="@+id/overviewInsurance" />

            <TextView
                android:id="@+id/overviewTicket"
                android:layout_width="match_parent"
                android:layout_height="wrap_content"
                android:padding="4dp"
                android:gravity="end"
                android:layout_marginEnd="24dp"
                android:text="1 000 000.00 zł"
                android:textColor="@color/teal_200"
                android:textSize="18sp"
                app:layout_constraintStart_toStartOf="parent"
                app:layout_constraintTop_toBottomOf="@+id/overviewInsurance" />

        </androidx.constraintlayout.widget.ConstraintLayout>
    </androidx.cardview.widget.CardView>

</LinearLayout>

</ScrollView>
```

W następnym kroku przygotujmy layouty dla `RecyclerView`
- przedstawiający sumaryczne koszty ze względu na kategorią poniesione dla każdego posiadanego samochodu
- przedstawiający podstawowe informacje o posiadanych samochodach

`ite_recyclerview_cost_overview.xml`
```xml
<?xml version="1.0" encoding="utf-8"?>
<androidx.cardview.widget.CardView xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    android:layout_width="match_parent"
    android:layout_height="wrap_content"
    android:layout_marginStart="40dp"
    android:layout_marginEnd="40dp"
    app:cardCornerRadius="45dp">

    <androidx.constraintlayout.widget.ConstraintLayout
        android:layout_width="match_parent"
        android:layout_height="wrap_content"
        android:layout_marginStart="4dp"
        android:layout_marginEnd="4dp"
        android:background="@color/dark_blue_500">

        <TextView
            android:id="@+id/overviewRecyclerViewCarName"
            android:layout_width="match_parent"
            android:layout_height="wrap_content"
            android:background="@color/green_500"
            android:gravity="center"
            android:padding="4dp"
            android:text="@string/car_name"
            android:textColor="@color/black"
            android:textSize="24sp"
            app:layout_constraintStart_toStartOf="parent"
            app:layout_constraintTop_toTopOf="parent" />

        <TextView
            android:layout_width="match_parent"
            android:layout_height="wrap_content"
            android:padding="4dp"
            android:layout_marginStart="24dp"
            android:text="@string/tankowanie"
            android:textColor="@color/teal_200"
            android:textSize="18sp"
            app:layout_constraintStart_toStartOf="parent"
            app:layout_constraintTop_toBottomOf="@+id/overviewRecyclerViewCarName" />

        <TextView
            android:id="@+id/overviewRecyclerViewFueling"
            android:layout_width="match_parent"
            android:layout_height="wrap_content"
            android:padding="4dp"
            android:gravity="end"
            android:layout_marginEnd="24dp"
            android:text="1 000 000.00 zł"
            android:textColor="@color/teal_200"
            android:textSize="18sp"
            app:layout_constraintStart_toStartOf="parent"
            app:layout_constraintTop_toBottomOf="@+id/overviewRecyclerViewCarName" />

        <TextView
            android:layout_width="match_parent"
            android:layout_height="wrap_content"
            android:padding="4dp"
            android:layout_marginStart="24dp"
            android:text="@string/serwis"
            android:textColor="@color/teal_200"
            android:textSize="18sp"
            app:layout_constraintStart_toStartOf="parent"
            app:layout_constraintTop_toBottomOf="@+id/overviewRecyclerViewFueling" />

        <TextView
            android:id="@+id/overviewRecyclerViewService"
            android:layout_width="match_parent"
            android:layout_height="wrap_content"
            android:padding="4dp"
            android:gravity="end"
            android:layout_marginEnd="24dp"
            android:text="1 000 000.00 zł"
            android:textColor="@color/teal_200"
            android:textSize="18sp"
            app:layout_constraintStart_toStartOf="parent"
            app:layout_constraintTop_toBottomOf="@+id/overviewRecyclerViewFueling" />

        <TextView
            android:layout_width="match_parent"
            android:layout_height="wrap_content"
            android:padding="4dp"
            android:layout_marginStart="24dp"
            android:text="@string/parkowanie"
            android:textColor="@color/teal_200"
            android:textSize="18sp"
            app:layout_constraintStart_toStartOf="parent"
            app:layout_constraintTop_toBottomOf="@+id/overviewRecyclerViewService" />

        <TextView
            android:id="@+id/overviewRecyclerViewParking"
            android:layout_width="match_parent"
            android:layout_height="wrap_content"
            android:padding="4dp"
            android:gravity="end"
            android:layout_marginEnd="24dp"
            android:text="1 000 000.00 zł"
            android:textColor="@color/teal_200"
            android:textSize="18sp"
            app:layout_constraintStart_toStartOf="parent"
            app:layout_constraintTop_toBottomOf="@+id/overviewRecyclerViewService" />

        <TextView
            android:layout_width="match_parent"
            android:layout_height="wrap_content"
            android:padding="4dp"
            android:layout_marginStart="24dp"
            android:text="@string/ubezpieczenie"
            android:textColor="@color/teal_200"
            android:textSize="18sp"
            app:layout_constraintStart_toStartOf="parent"
            app:layout_constraintTop_toBottomOf="@+id/overviewRecyclerViewParking" />

        <TextView
            android:id="@+id/overviewRecyclerViewInsurance"
            android:layout_width="match_parent"
            android:layout_height="wrap_content"
            android:padding="4dp"
            android:gravity="end"
            android:layout_marginEnd="24dp"
            android:text="1 000 000.00 zł"
            android:textColor="@color/teal_200"
            android:textSize="18sp"
            app:layout_constraintStart_toStartOf="parent"
            app:layout_constraintTop_toBottomOf="@+id/overviewRecyclerViewParking" />

        <TextView
            android:layout_width="match_parent"
            android:layout_height="wrap_content"
            android:padding="4dp"
            android:layout_marginStart="24dp"
            android:text="@string/mandat"
            android:textColor="@color/teal_200"
            android:textSize="18sp"
            app:layout_constraintStart_toStartOf="parent"
            app:layout_constraintTop_toBottomOf="@+id/overviewRecyclerViewInsurance" />

        <TextView
            android:id="@+id/overviewRecyclerViewTicket"
            android:layout_width="match_parent"
            android:layout_height="wrap_content"
            android:padding="4dp"
            android:gravity="end"
            android:layout_marginEnd="24dp"
            android:text="1 000 000.00 zł"
            android:textColor="@color/teal_200"
            android:textSize="18sp"
            app:layout_constraintStart_toStartOf="parent"
            app:layout_constraintTop_toBottomOf="@+id/overviewRecyclerViewInsurance" />

    </androidx.constraintlayout.widget.ConstraintLayout>
</androidx.cardview.widget.CardView>
```

`item_recyclerview_cars_overview.xml`
```xml
<?xml version="1.0" encoding="utf-8"?>
<androidx.cardview.widget.CardView xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    android:layout_width="wrap_content"
    android:layout_height="wrap_content"
    android:layout_marginStart="4dp"
    android:layout_marginEnd="4dp"
    app:cardCornerRadius="45dp">

    <androidx.constraintlayout.widget.ConstraintLayout
        android:layout_width="match_parent"
        android:layout_height="wrap_content"
        android:layout_marginStart="4dp"
        android:layout_marginEnd="4dp"
        android:background="@color/dark_blue_500">

        <TextView
            android:id="@+id/overviewViewPagerCarName"
            android:layout_width="200dp"
            android:layout_height="wrap_content"
            android:background="@color/green_500"
            android:gravity="center"
            android:padding="4dp"
            android:text="@string/domowy"
            android:textColor="@color/black"
            android:textSize="24sp"
            app:layout_constraintStart_toStartOf="parent"
            app:layout_constraintTop_toTopOf="parent" />

        <TextView
            android:id="@+id/textView"
            android:layout_width="match_parent"
            android:layout_height="wrap_content"
            android:layout_marginStart="24dp"
            android:padding="4dp"
            android:text="@string/marka"
            android:textColor="@color/teal_200"
            android:textSize="18sp"
            app:layout_constraintStart_toStartOf="parent"
            app:layout_constraintTop_toBottomOf="@+id/overviewViewPagerCarName" />

        <TextView
            android:id="@+id/overviewViewPagerBrand"
            android:layout_width="wrap_content"
            android:layout_height="wrap_content"
            android:layout_marginEnd="24dp"
            android:gravity="end"
            android:padding="4dp"
            android:text="@string/skoda"
            android:textColor="@color/teal_200"
            android:textSize="18sp"
            app:layout_constraintEnd_toEndOf="@+id/overviewViewPagerCarName"
            app:layout_constraintTop_toBottomOf="@+id/overviewViewPagerCarName" />

        <TextView
            android:layout_width="match_parent"
            android:layout_height="wrap_content"
            android:padding="4dp"
            android:layout_marginStart="24dp"
            android:text="@string/model"
            android:textColor="@color/teal_200"
            android:textSize="18sp"
            app:layout_constraintStart_toStartOf="parent"
            app:layout_constraintTop_toBottomOf="@+id/overviewViewPagerBrand" />

        <TextView
            android:id="@+id/overviewViewPagerModel"
            android:layout_width="match_parent"
            android:layout_height="wrap_content"
            android:padding="4dp"
            android:gravity="end"
            android:layout_marginEnd="24dp"
            android:text="@string/fabia"
            android:textColor="@color/teal_200"
            android:textSize="18sp"
            app:layout_constraintEnd_toEndOf="@+id/overviewViewPagerCarName"
            app:layout_constraintTop_toBottomOf="@+id/overviewViewPagerBrand" />

        <TextView
            android:layout_width="match_parent"
            android:layout_height="wrap_content"
            android:padding="4dp"
            android:layout_marginStart="24dp"
            android:text="@string/rok"
            android:textColor="@color/teal_200"
            android:textSize="18sp"
            app:layout_constraintStart_toStartOf="parent"
            app:layout_constraintTop_toBottomOf="@+id/overviewViewPagerModel" />

        <TextView
            android:id="@+id/overviewViewPagerYearOfProduction"
            android:layout_width="match_parent"
            android:layout_height="wrap_content"
            android:padding="4dp"
            android:gravity="end"
            android:layout_marginEnd="24dp"
            android:text="@string/_1999"
            android:textColor="@color/teal_200"
            android:textSize="18sp"
            app:layout_constraintEnd_toEndOf="@+id/overviewViewPagerCarName"
            app:layout_constraintTop_toBottomOf="@+id/overviewViewPagerModel" />

        <TextView
            android:layout_width="match_parent"
            android:layout_height="wrap_content"
            android:padding="4dp"
            android:layout_marginStart="24dp"
            android:text="@string/suma_koszt_w"
            android:textColor="@color/teal_200"
            android:textSize="18sp"
            app:layout_constraintStart_toStartOf="parent"
            app:layout_constraintTop_toBottomOf="@+id/overviewViewPagerYearOfProduction" />

        <TextView
            android:id="@+id/overviewViewPagerTotalCosts"
            android:layout_width="match_parent"
            android:layout_height="wrap_content"
            android:padding="4dp"
            android:gravity="end"
            android:layout_marginEnd="24dp"
            android:text="1 000.00 zł"
            android:textColor="@color/teal_200"
            android:textSize="18sp"
            app:layout_constraintEnd_toEndOf="@+id/overviewViewPagerCarName"
            app:layout_constraintTop_toBottomOf="@+id/overviewViewPagerYearOfProduction" />

    </androidx.constraintlayout.widget.ConstraintLayout>
</androidx.cardview.widget.CardView>
```

Zmieńmy formatowanie wartości zmiennoprzecinkowych, przejdźmy do pliku `FormatterUtil`

val decimalFormat = DecimalFormat("###,###.## zł")

In [ ]:
val decimalFormat = DecimalFormat("###,###.## zł")

Pierwszym będzie `OverviewCarsAdapter`

In [ ]:
class OverviewCarsAdapter : RecyclerView.Adapter<OverviewCarsAdapter.ViewHolder>() {

    private val cars = DataProvider.cars

    class ViewHolder(view: View) : RecyclerView.ViewHolder(view) {
        private val carNameTextView: TextView = view.findViewById(R.id.overviewViewPagerCarName)
        private val brandTextView: TextView = view.findViewById(R.id.overviewViewPagerBrand)
        private val modelTextView: TextView = view.findViewById(R.id.overviewViewPagerModel)
        private val yearTextView: TextView = view.findViewById(R.id.overviewViewPagerYearOfProduction)
        private val totalCostsTextView: TextView = view.findViewById(R.id.overviewViewPagerTotalCosts)

        fun bind (item: Car){
            carNameTextView.text = item.name
            brandTextView.text = item.brand
            modelTextView.text = item.model
            yearTextView.text = item.yearOfProduction.toString()
            totalCostsTextView.text = decimalFormat.format(item.costs.sumOf { it.amount }).toString()
        }
    }

    override fun onCreateViewHolder(parent: ViewGroup, viewType: Int): ViewHolder {
        return ViewHolder(
            LayoutInflater.from(parent.context).inflate(
            R.layout.item_recyclerview_cars_overview, parent, false))
    }

    override fun onBindViewHolder(holder: ViewHolder, position: Int) {
        val item = cars[position]
        holder.bind(item)
    }

    override fun getItemCount(): Int = cars.size
}

W funkcji `bind` pole `totalCostsTextView` będzie przechowywało informację o całkowitym poniesionym koszcie, aby to obliczyć na liście `costs` wywołujemy metodę `sumOf` - jako `selector` podajemy `amount`.

Przejdźmy do `OverviewCostsAdapter`, chcemy obliczyć sumę kosztów każdego typu i przekazać wynik do `TextView`. Napiszmy metodę pomocniczą w klasie `ViewHolder` zwracającą sumę kosztów o zadanym typie.

In [ ]:
private fun costValue(item: Car, costType: CostType): String{
    return decimalFormat.format(item.costs
        .filter { it.type == costType } // zwraca listę wszytkich kosztów o zadanym typie
        .sumOf { it.amount }).toString() // zwraca sumę wszystkich wartości
}

Pełna klasa adaptera:

In [ ]:
class OverviewCostsAdapter : RecyclerView.Adapter<OverviewCostsAdapter.ViewHolder>() {

    private val cars = DataProvider.cars

    class ViewHolder(view: View) : RecyclerView.ViewHolder(view) {
        private val carNameTextView: TextView = view.findViewById(R.id.overviewRecyclerViewCarName)
        private val refuelingTextView: TextView = view.findViewById(R.id.overviewRecyclerViewFueling)
        private val serviceTextView: TextView = view.findViewById(R.id.overviewRecyclerViewService)
        private val parkingTextView: TextView = view.findViewById(R.id.overviewRecyclerViewParking)
        private val insuranceTextView: TextView = view.findViewById(R.id.overviewRecyclerViewInsurance)
        private val ticketTextView: TextView = view.findViewById(R.id.overviewRecyclerViewTicket)

        fun bind (item: Car){
            carNameTextView.text = item.name
            refuelingTextView.text = costValue(item, CostType.REFUELING)
            serviceTextView.text = costValue(item, CostType.SERVICE)
            parkingTextView.text = costValue(item, CostType.PARKING)
            insuranceTextView.text = costValue(item, CostType.INSURANCE)
            ticketTextView.text = costValue(item, CostType.TICKET)
        }

        private fun costValue(item: Car, costType: CostType): String{
            return decimalFormat.format(item.costs
                .filter { it.type == costType }
                .sumOf { it.amount }).toString()
        }
    }

    override fun onCreateViewHolder(parent: ViewGroup, viewType: Int): ViewHolder {
        return ViewHolder(LayoutInflater.from(parent.context).inflate(
            R.layout.item_recyclerview_cost_overview, parent, false))
    }

    override fun onBindViewHolder(holder: ViewHolder, position: Int) {
        val item = cars[position]
        holder.bind(item)
    }

    override fun getItemCount(): Int = cars.size
}

Do metody `onViewCreated` klasy `OverviewFragment` dodajemy `RecyclerView`

In [ ]:
override fun onViewCreated(view: View, savedInstanceState: Bundle?) {
    super.onViewCreated(view, savedInstanceState)
    view.findViewById<RecyclerView>(R.id.overviewCostsRecyclerView).apply {
        layoutManager = LinearLayoutManager(context, LinearLayoutManager.HORIZONTAL, false)
        adapter = OverviewCostsAdapter()
    }

    view.findViewById<RecyclerView>(R.id.overviewCarsRecyclerView).apply {
        layoutManager = LinearLayoutManager(context, LinearLayoutManager.HORIZONTAL, false)
        adapter = OverviewCarsAdapter()
    }
}

Następnie chcemy dostać całkowitą sumę kosztów dla każdego typu (dla wszystkich samochodów), napiszmy metodę pomocniczą

In [ ]:
private fun totalValue(costType: CostType): String {
    return decimalFormat.format(DataProvider.cars
        .flatMap { it.costs }
        .filter { it.type == costType }
        .sumOf { it.amount }).toString()
}

Ponieważ każdy obierkt `Car` zawiera `List<Cost>`, wykonując `map` dostaniemy `List<List<Cost>>`. Będziemy sumować wszystkie koszty o zadanym typie, więc chcemy mieć jedną listę - w tym celu wykorzystujemy `flatMap`, która zwróci `List<Cost>`. Dalsze krokie są takie same jak poprzednio - wykonujemy `filter` aby dostać listę wszystkich kosztów ze względu na zadany typ, następnie `sumOf` zwraca sumę wszystkich wartości (`amount`).

In [ ]:
class OverviewFragment : Fragment() {

    override fun onCreateView(
        inflater: LayoutInflater, container: ViewGroup?,
        savedInstanceState: Bundle?
    ): View? {
        // Inflate the layout for this fragment
        return inflater.inflate(R.layout.fragment_overview, container, false)
    }

    override fun onViewCreated(view: View, savedInstanceState: Bundle?) {
        super.onViewCreated(view, savedInstanceState)
        view.findViewById<RecyclerView>(R.id.overviewCostsRecyclerView).apply {
            layoutManager = LinearLayoutManager(context, LinearLayoutManager.HORIZONTAL, false)
            adapter = OverviewCostsAdapter()
        }

        view.findViewById<RecyclerView>(R.id.overviewCarsRecyclerView).apply {
            layoutManager = LinearLayoutManager(context, LinearLayoutManager.HORIZONTAL, false)
            adapter = OverviewCarsAdapter()
        }

        view.findViewById<TextView>(R.id.overviewFueling).text = totalValue(CostType.REFUELING)
        view.findViewById<TextView>(R.id.overviewService).text = totalValue(CostType.SERVICE)
        view.findViewById<TextView>(R.id.overviewParking).text = totalValue(CostType.PARKING)
        view.findViewById<TextView>(R.id.overviewInsurance).text = totalValue(CostType.INSURANCE)
        view.findViewById<TextView>(R.id.overviewTicket).text = totalValue(CostType.TICKET)

    }

    private fun totalValue(costType: CostType): String {
        return decimalFormat.format(DataProvider.cars
            .flatMap { it.costs }
            .filter { it.type == costType }
            .sumOf { it.amount }).toString()
    }
}

Możemy przetestować aplikację

<table><tr><td><img src="https://media2.giphy.com/media/0lpf97nejk58XmgAP1/giphy.gif?cid=790b7611c2da1160570c8dd99550ec3cec763c3d4c38b025&rid=giphy.gif&ct=g" width="150" /></td><td><img src="https://media2.giphy.com/media/hv5Ewg1qBwSj99Ju82/giphy.gif?cid=790b7611ccbb9fccb551bafa99c524b39c01a03d957a334b&rid=giphy.gif&ct=g" width="150" /></td><td><img src="https://media4.giphy.com/media/FkSzT1Y6TNOtg89XKr/giphy.gif?cid=790b76119c5e65885f7be2c85549b516585eebeb4a8d4baa&rid=giphy.gif&ct=g" width="150" /></td></tr></table>